# Tutorial 2: Natural Hazard Risk Assessment

In the second tutorial of this week, we are going to use publicly available hazard data and exposure data to do a risk assessment for an area of choice within Europe. More specifically we will look at damage due to wind storms and flooding. 

We will use both Copernicus Land Cover data and OpenStreetMap to estimate the potential damage of natural hazards to the built environment. We will use Copernicus Land Cover data to estimate the damage to specific land-uses, whereas we will use OpenStreetMap to assess the potential damage to the road system.

## Learning Objectives
<hr>

- To know how to download data from the Copernicus Climate Data Store using the `cdsapi` and access it through Python.
- To be able to open and visualize this hazard data.
- To know how to access and open information from the Copernicus Land Monitoring System. Specifically the Corine Land Cover data.
- To understand the basic approach of a natural hazard risk assessment.
- To be able to use the `DamageScanner` to do a damage assessment.
- To interpret and visualize the damage estimates.

<h2>Tutorial Outline<span class="tocSkip"></span></h2>
<hr>
<div class="toc"><ul class="toc-item">
<li><span><a href="#1.-Introducing the packages" data-toc-modified-id="1.-Introducing-the-packages-2">1. Introducing the packages</a></span></li>
<li><span><a href="#2.-Downloading and accessing natural hazard data" data-toc-modified-id="2.-Extracting-hazard-data-3">2. Downloading and accessing natural hazard data</a></span></li>
<li><span><a href="#3.-Explore the natural hazard data for your area of interest" data-toc-modified-id="3.-Explore-hazard-data-4">3. Explore the natural hazard data for your area of interest</a></span></li>
<li><span><a href="#4.-Downloading and accessing Corine Land Cover" data-toc-modified-id="4.-Extracting-CLC-data-3">4. Downloading and accessing Corine Land Cover</a></span></li>    
<li><span><a href="#5.-Perform a damage assessment for your area of interest using Coring Land Cover" data-toc-modified-id="5.-Damage-CLC-5">5. Perform a damage assessment for your area of interest using Coring Land Cover</a></span></li>
<li><span><a href="#5.-Perform a damage assessment for your area of interest using OpenStreetMap" data-toc-modified-id="5.-Damage-OSM-5">6. Perform a damage assessment for your area of interest using OpenStreetMap</a></span></li>    
<li><span><a href="#7.-Interpret and visualize your damage estimates" data-toc-modified-id="5.-Analyze-buildings-6">7. Interpret and visualize your damage estimates</a></span></li></ul></div>

## 1.Introducing the packages
<hr>

Within this tutorial, we are going to make use of the following packages: 

[**GeoPandas**](https://geopandas.org/) is a Python packagee that extends the datatypes used by pandas to allow spatial operations on geometric types.

[**OSMnx**](https://osmnx.readthedocs.io/) is a Python package that lets you download geospatial data from OpenStreetMap and model, project, visualize, and analyze real-world street networks and any other geospatial geometries. You can download and model walkable, drivable, or bikeable urban networks with a single line of Python code then easily analyze and visualize them. You can just as easily download and work with other infrastructure types, amenities/points of interest, building footprints, elevation data, street bearings/orientations, and speed/travel time.

[**xarray**](https://docs.xarray.dev/) is a Python package that allows for easy and efficient use of multi-dimensional arrays.

[**Matplotlib**](https://matplotlib.org/) is a comprehensive Python package for creating static, animated, and interactive visualizations in Python. Matplotlib makes easy things easy and hard things possible.

*We will first need to install the missing packages in the cell below. Uncomment them to make sure we can pip install them*

In [ ]:
#!pip install osmnx
#!pip install geopandas
#!pip install contextily

And we will import these packages in the cell below:

In [27]:
import os
import numpy as np
import rasterio
import cdsapi
import xarray as xr
import pygeos 
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt

from zipfile import ZipFile
from matplotlib import rcParams, cycler

import urllib3
urllib3.disable_warnings()

## 2. Downloading and accessing natural hazard data
<hr>

We are going to perform a damage assessment using both windstorm data and flood data for Europe.

The windstorm data will be downloaded from the [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/). As we have seen during the lecture, and as you can also see by browsing on this website, there is an awful lot of climate data available through this Data Store. As such, it is very valuable to understand how to access and download this information to use within an analysis. To keep things simple, we only download one dataset today: [A winter windstorm](https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-european-wind-storm-indicators?tab=overview). 

We will do so using an **API**, which is the acronym for application programming interface. It is a software intermediary that allows two applications to talk to each other. APIs are an accessible way to extract and share data within and across organizations. APIs are all around us. Every time you use a rideshare app, send a mobile payment, or change the thermostat temperature from your phone, you’re using an API.

However, before we can access this **API**, we need to take a few steps. Most importantly, we need to register ourselves on the [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/) portal. To do so, we need to register, as explained in the video clip below:

<img src="../_static/images/CDS_registration.gif" class="bg-primary mb-1">

Now, the next step is to access the API. You can now login on the website of the [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/). After you login, you can click on your name in the top right corner of the webpage (next to the login button). On the personal page that has just opened, you will find your user ID (**uid**) and your personal **API**. You need to add those in the cell below to be able to download the windstorm.

As you can see in the cell below, we download a specific windstorm that has occured on the fifth of December in 2013. This is storm Xaver. 

In [34]:
uid = 76974
apikey = 'ace04e3e-84e3-4d0c-ae9d-d248aca1f5e5'

c = cdsapi.Client(key=f"{uid}:{apikey}", url="https://cds.climate.copernicus.eu/api/v2")

c.retrieve(
    'sis-european-wind-storm-indicators',
    {
        'variable': 'all',
        'format': 'zip',
        'product': 'windstorm_footprints',
        'year': '2013',
        'month': '12',
        'day': '05',
    },
    'Xaver.zip')

2022-12-15 11:16:14,182 INFO Welcome to the CDS
2022-12-15 11:16:14,184 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-european-wind-storm-indicators
2022-12-15 11:16:14,268 INFO Request is completed
2022-12-15 11:16:14,270 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data0/dataset-sis-european-wind-storm-indicators-3532b693-f64c-4771-8a00-4bfaace9c079.zip to Xaver.zip (14.4M)
2022-12-15 11:16:15,981 INFO Download rate 8.4M/s                                                                      


Result(content_length=15087991,content_type=application/zip,location=https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data0/dataset-sis-european-wind-storm-indicators-3532b693-f64c-4771-8a00-4bfaace9c079.zip)

In [3]:
with ZipFile('download.zip') as zf:
    for file in zf.namelist():
        with zf.open(file) as f:
            ds_disk = xr.open_dataset(f)

In [4]:
df = ds_disk.to_dataframe()
df = df.dropna()

In [8]:
gdf = gpd.GeoDataFrame(df.reset_index().drop(columns=['y','x','longitude','latitude']))

In [9]:
gdf = gdf.set_crs('epsg:4326')

In [10]:
place_name = "Nieuwland, Amersfoort, The Netherlands"
area = ox.geocode_to_gdf(place_name)

## 3. Explore the natural hazard data for your area of interest
<hr>

In [ ]:
zd

## 4. Download and access Copernicus Land Cover data
<hr>

Unfortunately, there is no API option to download the [Corine Land Cover](https://land.copernicus.eu/pan-european/corine-land-cover) data. We will have to download the data from the website first.

To do so, we will first have to register ourselves again on the website. Please find in the video clip below how to register yourself on the website of the [Copernicus Land Monitoring Service](https://land.copernicus.eu/):

<img src="../_static/images/CLMS_registration.gif" class="bg-primary mb-1">

Now click on the Login button in the top right corner to login on the website. There are many interesting datasets on this website, but we just want to download the Corine Land Cover data, and specifically the latest version: [Corine Land Cover 2018](https://land.copernicus.eu/pan-european/corine-land-cover/clc2018?tab=download). To do so, please select the **Corine Land Cover - 100 meter**. Now click on the large green Download button. Your download should start any minute.

Slightly annoying, the file you have downloaded is double zipped. Its slightly inconvenient to open this through Python and within Google Drive. So let's unzip it twice and store the data in a directory named `Data`. 

In [14]:
CLC_location = os.path.join('C:\Data\CLC\DATA','U2018_CLC2018_V2020_20u1.tif')

In [17]:
CLC_location

'C:\\Data\\CLC\\DATA\\U2018_CLC2018_V2020_20u1.tif'

In [19]:
with rasterio.open(CLC_location) as src:
    landuse = src.read()[0, :, :]
    transform = src.transform

2022-12-15 10:53:07,413 WARNING CPLE_AppDefined in The definition of projected CRS EPSG:3035 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.


## 5. Perform a damage assessment for your area of interest using Coring Land Cover
<hr>

## 6. Perform a damage assessment for your area of interest using OpenStreetMap
<hr>

## 7. Interpret and visualize your damage estimates
<hr>